# lpmod

Demonstrates the use of advanced starts in LP.   
Reads a continuous model from a file, solves it, and then modifies one variable bound.   
The resulting model is then solved in two different ways: starting from the solution of the original model, or restarting from scratch.

In [1]:
#!/usr/bin/env python3.7

# Copyright 2020, Gurobi Optimization, LLC

# This example reads an LP model from a file and solves it.
# If the model can be solved, then it finds the smallest positive variable,
# sets its upper bound to zero, and resolves the model two ways:
# first with an advanced start, then without an advanced start
# (i.e. 'from scratch').

import sys
import gurobipy as gp
from gurobipy import GRB

In [2]:
if len(sys.argv) < 2:
    print('Usage: lpmod.py filename')
    quit()

In [9]:
sys.argv[2]

'/Users/yj/Library/Jupyter/runtime/kernel-850e576c-e816-4b9d-a614-5dd03b27bf8b.json'

In [13]:
# Read model and determine whether it is an LP

model = gp.read(sys.argv[2])
if model.isMIP == 1:
    print('The model is not a linear program')
    sys.exit(1)

status = model.status

In [ ]:
if status == GRB.INF_OR_UNBD or status == GRB.INFEASIBLE \
   or status == GRB.UNBOUNDED:
    print('The model cannot be solved because it is infeasible or unbounded')
    sys.exit(1)

if status != GRB.OPTIMAL:
    print('Optimization was stopped with status %d' % status)
    sys.exit(0)

In [ ]:
# Find the smallest variable value

minVal = GRB.INFINITY
for v in model.getVars():
    if v.x > 0.0001 and v.x < minVal and v.lb == 0.0:
        minVal = v.x
        minVar = v

print('\n*** Setting %s from %g to zero ***\n' % (minVar.varName, minVal))
minVar.ub = 0.0

In [ ]:
# Solve from this starting point
model.optimize()

# Save iteration & time info
warmCount = model.IterCount
warmTime = model.Runtime

In [ ]:
# Reset the model and resolve
print('\n*** Resetting and solving without an advanced start ***\n')
model.reset()
model.optimize()

clodCount = model.IterCount
coldTime = model.Runtime

In [ ]:
print('')
print('*** Warm start %g iteration, %g seconds' % (warmCount, warmTime))
print('*** Cold start %g iteration, %g seconds' % (clodCount, coldTime))